I. C++ basics

1.0 preprocessor, compiler, link

http://www.runoob.com/cplusplus/cpp-preprocessor.html


![](img/comiler_process.png_)

In [1]:
#include <string>
#include <vector>
#include <iostream>
#include <algorithm>
#include <memory>
#include <functional>
#include <map>
#include <mutex> 
using namespace std;

1.1 Declaration, definition and initialization 

* Declaration: 声明式告诉编译器某个东西的名称和类型，但略去细节

In [2]:
extern int x;

* extern ?

In [3]:
std::size_t numDigits(int number);

In [7]:
class Widget;

In [5]:
template<typename T>

* Definition:
定义式的任务是提供编译器一些声明式遗漏的细节。对对象而言，定义式是编译器为对象拨发内存的地点；对function或function template而言，定义式提供了代码本体。对class或class template而言，定义式列出他们的成员

In [6]:
std::size_t numDigits(int number){
    std::size_t digits = 1;
    
    while((number/=10)!=0) ++digits;
    return digits;
}

In [1]:
class Widget{
public:
    Widget(){};
    ~Widget(){};
};

尽量延后定义，需要使用变量时再定义  Term 26

In [4]:
std::string encrypt(const std::string& s){
    std::string encrypted;
    
    if(!s.length())
        throw logic_error("invalid string to encrypt");
    
    return encrypted;
}

如果报了异常，则encrypted的定义就浪费了，白白构造和析构

赋值操作构造和析构比节约  Term 26

方案1，1次构造，1次析构，10次赋值

方案2，10次构造，10次析构。

在意效率选1

In [7]:
Widget w;
for(int i=10; i>0; --i){
    // w=xx
}

In [8]:
for(int i=10; i>0; --i){
    Widget w;
}

* Initialization: 初始化是“给予对象初值”的过程。default构造函数是一个可被调用而不带任何实参，不然没有参数，不然每个参数都有缺省值

确定对象被使用前已被初始化   Term 4

1) 对于类来说
* 确保所有对象初始化，即便是int类型，有的编译器不保证初始化
* 用初始化列表
* 在继承的语境中，用初始化列表初始化父类
* 对于const或reference，他们不能被赋值，必须用初始化列表初始化

初始化参数列表

初始化类的成员有两种方式，一是使用初始化列表，二是在构造函数体内进行赋值操作。

对于对象类型，用初始化列表更高效，因为省去了调用默认构造函数的过程，直接使用拷贝构造函数。必须使用初始化列表的情况:
* 常量成员，因为常量只能初始化不能赋值，所以必须放在初始化列表里面
* 引用类型，引用必须在定义的时候初始化，并且不能重新赋值，所以也要写在初始化列表里面
* 没有默认构造函数的类类型，因为使用初始化列表可以不必调用默认构造函数来初始化，而是直接调用拷贝构造函数初始化。

In [8]:
class A {
private:
    int value;
    
public:
    A(int value):value(value) {};
};

class B: public A {
private:
    bool flag;
    
public:
    explicit B(int x = 0, bool b = true):A(x), flag(b){}; 
};

In [9]:
B bObj;
B bObj1(1, false);

2) non-local static 对象的初始化 ??

3) move constructor  ??

* c++11开始支持，如果没有移动，则必须用拷贝，如果对应大数组，则牵扯到大量资源的复制
* IO 类和 unique_ptr 不支持资源共享，不能拷贝，只能移动

In [2]:
class IntVec{
public:
    vector<int> elements;
    
    IntVec(vector<int>& list):elements(list){}
    IntVec(IntVec &&s) noexcept:elements(s.elements) {};
}

In [3]:
vector<int> v = {1,2,3,4};
IntVec v1(v);
IntVec v2(std::move(v1));

1.2 隐式类型转换

关于单参构造函数和类型转换函数。单参构造函数容易被误用

其实单参构造函数就是只有一个参数的构造函数，拷贝构造函数就是他的一种，单参构造函数和类型转化函数用法正好相反，单参构造函数是把形参中的值或者是对象转化为本对象，而类型转化函数是把本对象转换为一个值或者一个其他对象

In [10]:
long a_long = 10;
int a_int=a_long;     //long->int

In [11]:
class D {
public:
    std::string id;
    D(std::string id):id(id){};
}

In [12]:
D aWrongConversion=std::string("abc")  //这里就做了隐式类型转换，把string转换成class D.

(D &) @0x7f043a1c0038


* explicit 关键字修饰单参构造函数可以解决这个问题

In [13]:
class C {
public:
    std::string value;
    
    explicit C(std::string x):value(x){};
};

In [14]:
//C conversionWontHappen = std::string("abc");    
// candidate constructor (the implicit move constructor) not viable

1.3 Static, local static, non-local static

static对象，寿命从构造出来开始到程序结束为止，不存储在stack和heap。包括：
* global对象
* namesapce作用域中对象
* class, function, file作用域内被定义为static对象
* function内定义的static为local static对象，其他为non-local static对象
* 所有static对象在main()结束时调用其析构函数被销毁

1.4 复合类型（compound type） reference and pointer

reference, lvalue reference, rvalue reference

引用，主要指代左值引用, lvalue reference

1) 引用只能初始化，不能赋值，因为c++不允许引用改变指向对象

In [15]:
int iVal = 1024;
int& refVal = iVal;

In [16]:
// int& failRefVal;  
//declaration of reference variable 'failRefVal' requires an initializer

2) 引用**不是对象**，而是对象的一个别名，所以不存在引用的引用。

3) 引用必须指向一个对象！

In [17]:
// int &refVal1 = 10; 
//non-const lvalue reference to type 'int' cannot bind to a temporary of type 'int'

4) rvalue reference

左值引用不能绑定表达式，字面常量，右值引用正相反，只能绑定临时的，不能绑定左值

In [18]:
int &&rr = 10;

5) 右值用于移动操作move，左值用于拷贝

const pointer vs pointer to const

1) const pointer, 指针是常量，不能改变值

In [19]:
int a = 10, b = 20;
int * const  pInt = &a;

In [20]:
//pInt = &b;  //variable 'pInt' declared const here

In [21]:
a = 20;
*pInt

(int) 20


2) pointer to const, 指向的是常量，不能通过指针修改其值，但是可以直接改被指向值

In [22]:
const int * cpInt = &a;

In [23]:
a = 30;
*cpInt

(const int) 30


In [24]:
//*cpInt = 20;  //read-only variable is not assignable

Pass-by-value,  Pass-by-reference           （Term 20)

* 对于内置类型，int，enum，... 值传递更高效
* 对于用户定义类，引用传值更高效, 避免了值传递中的复制操作
* pass-by-reference-to-const 还可以避免slicing（对象切割）的问题

当传递的时derived class时，拷贝构造函数被调用，原本属于derived class的function， member被切割

In [69]:
// bool a_interface_accept_base_class(Base b){} ;    

In [70]:
// bool a_interface_accept_base_class(const Base& b){} ;     
//传递的是地址，copy-constructor未调用

Pass-by-reference-to-const

reference to const - 对常量的引用， 简称常量引用

Reference to pointer

1.5 Statements and flow control

1. loop

In [27]:
vector<int> nums = {1,2};

* Range-based for loop

The for-loop has another syntax, which is used exclusively with ranges:

for ( declaration : range ) statement;

Ranges are sequences of elements, including arrays, containers, and any other type supporting the functions __begin()__ and __end()__; 

In [28]:
for (int i: nums){
    cout << i << endl;
}

1
2


User prefix increment, in for loop, extra copy involved with the postfix. So prefix can be faster

In [29]:
for (int i=0; i<nums.size(); ++i){
    cout << nums[i];
}

* iterator

In [30]:
for (vector<int>::iterator it=nums.begin(); it!=nums.end(); ++it){
    *it += 1;  
    cout << *it << endl;
}

122
3


* begin return const_iterator if vector object is const-qualified

In [31]:
const vector<int> cNums = {1,2};    

for (auto it=cNums.begin(); it!=cNums.end(); it++){
    // *it += 1;  //function 'operator*' which returns const-qualified type
    cout << *it;
}

2. switch case

In [32]:
int condition = 1;

In [33]:
switch(condition){
    case 1:
    case 2:
        cout << "condition is 1 or 2"<< endl;
    default:
        cout << "condition did not match";
}

12condition is 1 or 2


1. 6 Dynamic Memory 

* 用处：
    * C++ 是by default是pass-by-value，面对vector等大的存储结构，很低效，如何把local vector传递出去？不要在stack中建立数据，建立动态内存，在堆中创建
* new，delete直接管理堆，会产生很多潜在问题。引入智能指针，shared_ptr, weak_ptr, unique_ptr来管理。

1) 变量作用域：
* 全局变量
* 局部变量
* static
* 动态分配: 动态分配的对象的生命周期与在哪里创建无关，只有显示释放才会被销毁
    
2) 内存类型：
* 栈内存-- 函数中非static  （全局变量??）
* static内存 -- static变量
* heap-堆 -- 动态分配   

new and delete

* new

nothrow, if failed to allocate memory, will not throw bad alloc，but return nullptr

In [34]:
int *dp;
dp = new int;

In [35]:
dp = new (nothrow) int[5];

In [36]:
if(dp == nullptr){
    cout << "Error: memory could not be allocated" << endl;
}

In [37]:
*dp = 10;

* delete

In [38]:
delete [] dp;

smart pointer

pointer-like object，类指针对象

底层封装了指针的对象, 注意是对象，不是指针

* 资源：
    * 资源包括内存，文件描述器（file descriptors ??), mutex locks， db connections， socket
* 对资源的操作：
    * 创建，初始化，销毁，移动，拷贝
    * 创建和初始化要同时 RAII

RAII  < Term 13>

* Resource Acquisition Is Initialization 初始化时获取资源
* 对象析构时候要正确释放资源

copy < Term 14 think carefully about copying behavior in resource-managing classes>

In [2]:
class Lock {
private:
    mutex* mutexPtr;
public:
    explicit Lock(mutex* pm): mutexPtr(pm) { lock(mutexPtr);}
    ~Lock(){ unlock(mutexPtr); }

    void lock(mutex* pm) {};
    void unlock(mutex* pm) {};
};

?   


In [6]:
{
    mutex m;
    Lock m1(&m);
    Lock m2(m1);    // 拷贝发生，你想如何定义这个拷贝
}

* 当一个 RAII 对象被复制，会发生什么事？
    * 禁止复制 -- 不允许资源共享
    * 对底层资源使用“引用计数法” reference-count -- 允许资源共享
    * 复制底层资源 -- 做一个深拷贝
    * 转移底部资源拥有权 -- move

In [7]:
class SharedLock {
private:
    shared_ptr<mutex> mutexPtr;
public:
    explicit Lock(mutex* pm): mutexPtr(pm) { lock(mutexPtr.get());}
    ~Lock(){ unlock(mutexPtr); }

    void lock(mutex* pm) {};
    void unlock(mutex* pm) {};
};

?      


auto_ptr 

* c++11 deprecated, 被 unique_ptr 代替
* auto_ptr 保证当对象不在需要时自动清理
* 如果 auto_ptr 被复制，那么会造成资源多次删除，未定义行为。所以类似于 unique_ptr 不能被复制，和copy assignment

In [93]:
{
    int* p = new int(0);
    // auto_ptr<int> ap1 = p; // deprecated
    // auto_ptr<int> ap2 = p;
}

p 会被删除两次
* 所以 为了防止，通过 copy-constructor 或者 copy-assignment 赋值他们，他们会变成 null

In [94]:
// ap1 = ap2;              ap2 --> nullptr

* 又因为，auto_ptr 不支持赋值和拷贝，所以 stl 标准容器中不能用它

shared_ptr

* constructor

1) 初始化为空的指针

In [46]:
{ 
    shared_ptr<int> p; 
    if ( not p && p == nullptr)
        cout << "P is nullptr " << p << endl;
}

P is nullptr 0


2) make_shared 在动态内存中分配一个对象并初始化它， 返回此对象的shared_ptr

In [49]:
{ shared_ptr<vector<int>> p = make_shared<vector<int>>(); }

3) copy-constructed, reference count 引用计数会增加，

In [48]:
{ 
    auto p = make_shared<int>(47);
    auto q(p); 
    // get 函数返回 shared_ptr内部封装的指针，不要乱用
    cout << *q.get() << endl;    
}

47


4) use_count, 计算引用计数

In [3]:
{
    shared_ptr<vector<int> > pA(new vector<int>({1,2,3}));
    cout << "use_count " << pA.use_count() << endl;
    
    auto pB = pA;
    cout << "use_count " << pB.use_count() << endl;
}


use_count 1
use_count 2



5) new vs make_shared

In [54]:
{ 
    shared_ptr<vector<int>> p = make_shared<vector<int>>();
    shared_ptr<vector<int>> pA(new vector<int>({1,2,3}));
}

6) shared_ptr get获取裸指针，从而对其指向的堆中的值进行操作

可以当作指针一样操作

In [53]:
// *pA

7) shared_ptr in class

* delete
    * 引用计数为0自动析构
    * 对于一些管理资源的对象来说，e.g db connection, 释放资源可能有自己的函数, shared_ptr 第二个参数可以传递自己的删除器 deleter

In [88]:
{ 
    // connection c = connect(&d);
    // shared_ptr<connection> p(&c, end_connection)
}

智能指针陷阱 

1) 不要混用普通指针和智能指针

* error: no viable conversion from 'int *' to 'shared_ptr<int>'

In [2]:
int *x(new int(1));
// shared_ptr<int> ptr = x;

* 如下例，混用可能造成，指针被意外释放

In [3]:
void process(shared_ptr<int> x) {;}
process(shared_ptr<int>(x));
*x     // x 已经被释放

2) 不要使用 get 初始化另一个智能指针，或赋值

In [26]:
{
    shared_ptr<int> p(new int(1));
    int *q = p.get();
    {
        shared_ptr<int>(q);
    }
    // p 的内部指针已经悬挂
}

* 原则是 get 出来的裸指针，一定不能被删除！

3) 如果智能指针管理的资源不是new分配的，记住传递给他一个删除器，（上db connection)

unique ptr ??

In [42]:
unique_ptr<int[]> pU(new int[5]);

In [43]:
pU[0] = 10;

weak_ptr  ??

1.8 const                    (Term 2)

In [44]:
//char * const authorName1 = "Scott Meyers";  
// ISO C++11 does not allow conversion from string literal to 'char *const'
const char * const authorName = "Scott Meyers";

In [45]:
const std::string authoerName1("Scott Meyers");   // String is better then char *

* class专属常量

类的静态数据成员不属于类的任何一个对象，他们并不是在类初始化时被定义的。一般来说，我们不能在累的内部初始化静态成员，而必须在外部定义，和初始化。并且只能定义**一次**

In [ ]:
class CostEstimate {
private:
    static const double counter;              // 常量声明式
}
// const double CostEstimate::counter = 1.0;    // 常量定义，在.cpp文件

* 只有一个例外，const 整数类型，可以在类内设定初始值：

In [46]:
class GamePlayer {
private:
    static const int NumTurns = 5;           //常量声明式 不是定义式！
}

In [47]:
// const int GamePlayer::NumTurns;          // 已经有初值了，定义时不提供值，在.cpp内

* 有的编译器不支持，class static member 需要用enum hack

In [ ]:
class GamePlayerHack{
private:
    enum { NumTurns = 5};
    
    int scores[NumTurns];
}

* const 成员函数  ??

* constexpr ??

const vs volatility ??

1.9 string, c style character string

* string 最重要的标准库类型之一，表示可变长度字符串

In [51]:
string s1 = "A string example";      // 调用拷贝构造函数赋值
string s2("A different string");     // 调用构造函数

In [52]:
s1 < s2

(bool) false


* C style character string不是一种类型，而是为了表达和使用字符串形成的约定俗成的写法。以空字符('\0')来标识结束

In [53]:
#include "string.h"
char ca[] = {'c', '+', '+', '\0'}

(char [4]) "c++"


* 对string的操作

In [54]:
cout << strlen(ca) << endl;           // similar to s1.size()

3


In [55]:
const char ca1[] = "A string example";
const char ca2[] = "A different string";
//if (ca1 < ca2)  
//warning: array comparison always evaluates to a constant
strcmp(ca1, ca2) < 0        //"Similar to S1 < S2"

(bool) false


In [56]:
char ca3[100];
strcpy(ca3, ca2);    // Similar to S3 = S2
strcat(ca3, ca1);   // similar to S3 + S1

* string to C-string

In [57]:
const char * str = s1.c_str();

1.10 Friend

In [2]:
class WithFriend {
friend void checkName(const WithFriend& f);
private:
    std::string name;
public:
    WithFriend(const std::string& name):name(name){};
};

void checkName(const WithFriend& f){
    cout << f.name << endl;
}

In [4]:
WithFriend withFriend1("ABC");
checkName(withFriend1)      // No problem in accessing private memebers

1.11 Casting

c++中有一些是关联类型，所以相互之间可以转换。

Categorize
* explicit cast
* implicit conversion
    * 无需coder介入的转换
    * 算数转换(arithmetic conversion), e.g int to double
    * 数组到指针
    * 指针的转换
    * 转换为boolean, e.g if(a_value)
    * 转换为常量， e.g const int& j = i;
    * 类类型转换，通过写带有一个单参构造函数实现

explicit cast
* old-style casts
    * (T)expression
    * T(expression)
* c++-style casts: cast-name<type>(expression)
    * const_cast<T>( expression ): 移除常量性
    * dynamic_cast<T>( expression )
    * reintepret_cast<T>( expression )
    * static_cast<T>( expression)

1) const_cast

如果被引用的值为const，则const_cast之后仍旧不能改变值，只能改变引用的const属性

In [20]:
{
    int number = 10;
    const int& ref = number;
    
    const_cast<int&>(ref) = 5;   // after convert could change value
}

2) dynamic_cast

继承中进行(safe downcasting)，static_cast upcasting安全，但是down_casting因为时静态绑定时检查，无法查出动态的类别

dynamic非常妙，必须是动态绑定的对象的指针或者引用，才可以进行dynamic cast.

如果没有virtual function，没有动态绑定，则cast会有error  // ERROR is not polymorphic

In [9]:
{
    class A {
        virtual void f(){};  // 必须定义一个虚函数，让类进行动态绑定
    };
    
    class B:public A {};
    
    B b;
    A& ref = b;
       
    B& refToB = dynamic_cast<B&>(ref);
}

3) reinterpret_cast

不可移植

4) static_cast
* 静态类型转换，编译时进行的强制类型转换，除了const_cast其余两种都可以被static_cast代替
* 优于old-style 强制类型转换的一点：static_cast在编译时会进行类型检查，而强制转换不会。
* 将typed指针转换为void * verse vise

In [52]:
{
    double a=10.1, *pA=&a;
    
    static_cast<int>(a);
    static_cast<void>(pA);
    
    // static_cast from 'double *' to 'int *' is not allowed
    // static_cast<int*>(pA); 
}

Minimize casting Term 27

In [7]:
class Window {
public:
    virtual void onResize() { size = 0; }
    int size = 1001;
};

class SpecialWindow: public Window {
public:
    virtual void onResize() {
        static_cast<Window>(*this).onResize();
        // Window::onResize();            correct
    }
    
    int getSize() { return Window::size; }
    void blink() {}
};

class SpecialWindow2: public Window {};

static_cast 之后调用的不是本来SpecialWindow示例里的Window，而是产生的副本上调用Window::Resize

如果函数中修改变量值，e.g size，修改就会丢失

In [8]:
SpecialWindow sWindow;
sWindow.onResize();
sWindow.getSize()

(int) 1001


dynamic_cast 

Problem: 用一个数组，存储一个基类的各种子类。而后判断其类型，进行不同的处理

* 那么数组中只能存储指针，我们首选shared_ptr

In [9]:
typedef vector<shared_ptr<Window>> VPW;
VPW winPtrs;

In [10]:
winPtrs.push_back(shared_ptr<Window>(new SpecialWindow()));
winPtrs.push_back(shared_ptr<Window>(new SpecialWindow2()));

* 接着要进行类型判断，每个shared_ptr究竟是那种derived class，方法对其进行dynamic_cast

In [11]:
for (VPW::iterator iter =  winPtrs.begin(); iter!=winPtrs.end(); ++iter){
    if (SpecialWindow *sp = dynamic_cast<SpecialWindow*>(iter->get())){
        cout << "Got an SpecialWindow" << endl;
    }
    else if(SpecialWindow2 *sp = dynamic_cast<SpecialWindow2*>(iter->get())){
        cout << "Got an SpecialWindow2" << endl;
    }
}

Got an SpecialWindow
Got an SpecialWindow2


SpecialWindow *sp = dynamic_cast<SpecialWindow*>(iter->get());
    * iter->get() 获得shared_ptr的裸指针，是 Window *
    * dynamic_cast<SpecialWindow*>(Window*)  把window* 转化为 derived*

* 这种写法非常不可取，因为dynamic_cast很没有效率
    * 设计存在缺陷，既然放到一个数组，那么就应该只调用其公共接口
    * 改善，从新设计公共接口，如果有些实例不需实现，可以留空实现

II. Object-Oriented

2.0 constructor, copy constructor, copy assignment, deconstructor     Term 5

1) C++自动为每一个类生成这4个函数

In [3]:
// class Empty {};

相当于定义了：

In [1]:
class Empty {
public:
    Empty(){};
    Empty(const Empty&rhs) {};
    ~Empty(){};
    
    Empty& operator=(const Empty& rhs) = default;
};

In [3]:
Empty e1;
Empty e2 = e1;

2) 当成员有const，或者referenc，default copy assginment会失效

In [5]:
template<typename T>
class NamedObject {
public:
    NamedObject(std::string& name, const T& value):nameValue(name), objectValue(value){};
private:
    std::string& nameValue;
    const T objectValue;
};

In [6]:
std::string newDogName = "Peter";
NamedObject<int> newDog(newDogName, 10);

std::string oldDogName = "Satch";
NamedObject<int> oldDog(oldDogName, 26);


object of type 'NamedObject<int>' cannot be assigned because its copy assignment operator is implicitly deleted

In [8]:
// newDog = oldDog;    

因为name是一个string 左值引用，所以不能被更改，赋值，因此 NamedObject本身的赋值运算符重载被删除

同理，如果NamedObject本身包含const member，那么编译器定义的default copy assignment operator也会失败，因为常量不能更改

（而编译器给的缺省拷贝，赋值构造函数就是简单的将每个成员进行拷贝）

* solution，自己定义copy constructor, copy assignment if needed， 或者拒绝编译器自动生成的函数

3) =default 
c++11中，如果我们需要默认的行为，可以通过在参数列表后面加上 =default 来要求编译器生成

和其他函数一样，如果 =default 出现在类的内部，则默认构造函数时内联的，如果在外部，则不是内联的

Copy constructor, Copy assignment    <Term 12>

2.1 不想使用编译器自动生成的函数，就明确拒绝

将拷贝构造函数，copy assignment设为private，并且不定义

In [10]:
template<typename T>
class NamedObject2 {
public:
    NamedObject2(const T& value):objectValue(value){};
private:
    const T objectValue;
    
    NamedObject2(const NamedObject2 & rhs);            
    NamedObject2& operator=(const NamedObject2 & rhs);
};

2) 或者定义一个uncopyable base class

In [11]:
class Uncopyable{
protected:
    Uncopyable(){};
    ~Uncopyable() {};

private:
    Uncopyable(const Uncopyable&);
    Uncopyable& operator=(const Uncopyable&);
}

?   


In [12]:
class UncopyableDerived: protected Uncopyable{};

In [14]:
UncopyableDerived aUncopyable;

In [16]:
// call to implicitly-deleted copy constructor of 'UncopyableDerived'
// UncopyableDerived anotherUncopyable(aUncopyable); 

2.2 继承

1) 基类定义

基类必须被定义，而不仅仅是声明，否则派生类不知道如何继承

In [3]:
class BaseOnlyDeclare;

//class Derived : BaseOnlyDeclare {}; //  base class has incomplete type

In [10]:
class Base {
public:
    Base() = default;                         
    Base(int value):value(value) {};
    
    virtual std::ostream& print (std::ostream& os) const {       //const member functions
        os << "Base print() " << value << std::endl;
        return os;
    }
    
    friend std::ostream& operator<<(std::ostream& os, const Base& b){
        return b.print(os);          // 这里的b对象，是动态绑定的，调用不同的print 函数
    }
    
    virtual ~Base() = default;                 // 析构函数必须 virtual

protected:
    int value;
}

?   


2) 派生类继承并初始化基类

在参数化列表中初始化基类

In [11]:
class Derived: public Base {
public:
    Derived(int value): Base(value) {};
    
    virtual std::ostream& print(std::ostream& os) const {
        os << "Derived print() " << value << std::endl;
        return os;
    }
}

3) 虚函数与动态绑定，见 2.3

4) override 关键字

C++11 可以使用override 来说明派生类中的虚函数

In [1]:
class BaseOverride {
public:
    virtual void f1(int) const;
    virtual void f2();
    void f3();
}

In [4]:
class DerivedOverride: public BaseOverride{
public:
    void f1(int) const override;
    // void f2(int) override;   // error f2 signiture no match
    // void f3() override;      //error, f3 not virtual
    // void f4() override;     //error, no f4 in base
}

5) final 关键字

In [17]:
class BaseFinal final {};

**error**: base 'BaseFinal' is marked 'final'

In [19]:
// class DeriveFromFinal: public BaseFinal {};

6) 继承与静态成员

In [1]:
class BaseStatic{
public:
    static void statmem();
};

class DerivedStatic: public BaseStatic{
    void f(const DerivedStatic&);
}

In [2]:
void DerivedStatic::f(const DerivedStatic& obj){
    BaseStatic::statmem();    //正确，通过基类访问
    DerivedStatic::statmem();  //正确，派生类可以访问基类静态成员
    obj.statmem();  //正确，通过 DerivedStatic 对象访问
    statmem();  //正确，通过 this 访问
}

2.2.1 Dynamic Binding  动态绑定

In [12]:
Base base_instance(10);
Derived derived_instance(11);

In [13]:
std::cout << base_instance << derived_instance;

Base print() 10
Derived print() 11


1） Dynamic binding, bind Derived to Base reference

动态绑定只有在指针，引用才会发生，实际的对象永远都是其对应的类型，进行静态绑定

In [14]:
Base& bp = derived_instance;    // Derived绑定到了Base reference上
bp.print(std::cout);

Derived print() 11


2） Base class cannot bind to Derived pointer/reference

non-const lvalue reference to type 'Derived' cannot bind to a value of unrelated type 'Base'

In [15]:
// Derived& dp = b;  

3) 强行不进行动态绑定

通常是当派生类覆盖了基类的版本时，通常情况下，基类的版本需要完成一些共同任务，强行调用基类版本

In [17]:
bp.Base::print(cout);

Base print() 11


2.2.2 析构函数必须 virtual    <Term 7>

1) 对于有多态性的继承，编译器需要知道每一个类该如何被析构，所以需要定义deconstructor virtual

2) 不要随便继承没有析构函数的类，包括所有STL容器，vector, list, set, tr1::unordered_map, string...

这些classes不是作为base classes，或者作为base calsses但不具备多态性

In [2]:
template<typename T>
class UserVector: public std::vector<T>{};

2.2.3 绝不在构造和析构过程中调用virtual  <Term 9>

**Warning** call to pure virtual member function 'log' has undefined behavior; overrides of 'log' in subclasses are not available in the constructor of 'BaseConstructorCallVirtual'

当base class构造的时候，virtual 函数的override 还没有定义呢。没有办法进行动态调用

In [1]:
class BaseConstructorCallVirtual{
public:
    BaseConstructorCallVirtual(){ /*log();*/ }
    virtual void log() const = 0;
    // 即使不定义为 pure virtual函数，如果没有定义，链接会报错
    // 即使定义，程序会调用基类的版本，无法实现多态
}

class DerivedConstructorCallVirtual: public BaseConstructorCallVirtual{
public:
    DerivedConstructorCallVirtual():BaseConstructorCallVirtual() {}
    virtual void log() const override {};
}

* Solution, 不定义为virtual，而是把构造期间需要的数据，由Derived 向上传递给 Base

In [4]:
class BasePassDataUp{
public:
    BasePassDataUp(std::string& logInfo){ log(logInfo); }
    void log(std::string& logInfo) const;
}

class DerivedPassDataUp: public BasePassDataUp{
public:
    DerivedPassDataUp(std::string& logInfo):BasePassDataUp(logInfo) {};
}

2.3 设计

2.3.1 Avoid returning "handles" to object internals Term 28

In [2]:
class Point {
public:
    Point(int x=0, int y=0):x(x), y(y) {}
    int x;
    int y;
}

In [3]:
struct RectData{
    Point ulhc;
    Point lrhc;
}

In [4]:
class Rectangle {
private:
    shared_ptr<RectData> pData;
    
public:
    Rectangle(shared_ptr<RectData>& pData):pData(pData) { }
    
    Point& upperLeft() const { return (*pData).ulhc; }
    const Point& lowerRight() const { return (*pData).lrhc; }
};
    
const Point& foo(){
    shared_ptr<RectData> pData(new RectData({Point(1,1), Point(2,2)}));
    Rectangle r(pData);
    const Point& dangle = r.lowerRight();
    cout << "before rect destoried " << dangle.x << endl;
    return dangle;
}


In [5]:
const Point& dangle = foo();

before rect destoried 2


In [6]:
dangle.x

(const int) 115939584


Rectangle(const Point& ulhc, const Point& lrhc) :pData(make_shared<RectData>()){ }
??
    
* upperLeft()返回了底层数据的引用。所有的这些引用，指针，iter都叫handles。 upperLeft()这样做破坏了数据的封装性
* lowerRight() 返回一个const ref，解决了封装性问题，但是，可能会造成dangle ref。

III. Exception

3.1 不要让异常跑出析构函数

容易造成析构不完全，内存泄漏。尤其是有list，第一个析构异常，后面都析构不了

3.2 异常安全

Strive for exception-safe code -- Term 29

异常安全，是保证在异常发生时，不会发生：
* 不会数据泄露，包括内存，死锁，数据库连接等等
* 不允许数据败坏

In [2]:
class Image {
public:
    Image(std::istream& imgSrc){}
}

class PrettyMenu {
public:
    void changeBackground(std::istream& imgSrc);
private:
    mutex mutex;
    Image* bgImage;
    int imageChange;
}

In [3]:
void PrettyMenu::changeBackground(std::istream& imgSrc) {
    //lock(&mutex);
    delete bgImage;
    ++imageChange;
    bgImage = new Image(imgSrc);
    //unlock(&mutex);
    
}

* 如果new image有异常，那么unlock不会被调用，mutex未被合理释放造成死锁
    * Lock m(&mutex) 通过析构函数释放，保证资源未被泄露
* 如果new image异常，但此时imageChange 已经 ++, 这叫数据败坏

3.3 多线程

IV. Operator overloading ??

* 重载运算符是具有**特殊名字**的函数，opeartor=
* 和其他函数一样，也包含返回类型，参数列表，函数体

In [1]:
class MyInteger {
private:
    int value;

public:
    MyInteger(int value=0):value(value){};
    
    MyInteger& operator=(const MyInteger& rhs){
        this->value = rhs.value;
        return *this;
    }
}

?   


* 用操作符调用，和直接调用operator= function等价

In [3]:
MyInteger integer(10), integer1;

integer1 = integer;
integer1.operator=(integer);

4.1 一元运算符

4.2 二元运算符

4.3 operator= 和 copy assignment，copy constructor

令operator= 返回一个 reference to *this  <Term 10>

为了实现连锁赋值，所有赋值运算符的重载，都应该返回 reference to *this, 包括 +=, -=,

In [5]:
MyInteger myX, myY, myZ;
myX = myY = myZ = MyInteger(10);

在 operator= 中处理"自我赋值"   <Term 11>

In [6]:
myX = myX;

* 因为别名, reference/pointer 引起的不明显的自我赋值

In [8]:
MyInteger& refMyInt = myX;
myX = refMyInt;

* 如果赋值中包含对指针，资源的操作，自我赋值会引发问题

先delete 了自己的内存，然后试图赋值给自己

1) solution 1: identity test

if（this == &rhs) return *this;

In [1]:
class Bitmap{};

class Widget{
private:
    Bitmap *p;     //指针，指向一个 heap 分配的对象

public:
    Widget& operator=(const Widget& rhs){
        delete p;
        p = new Bitmap(*rhs.p);
        return *this;
    }
}


?   


2) solultion 2: copy and swap

每一次赋值都做一个比较，效率低
```
    Widget temp(rhs);
    swap(temp);
    return *this;
```
拷贝，swap 比比较效率高？

v Template
* function template specilization
* class tempalte partial speicilization

VI Functional

6.0 可调用对象

* 函数
* 函数指针
* lambda表达式
* bind创建的对象 ??
* 函数对象 function object

In [8]:
int add(int i, int j) { return i+j; }

In [7]:
auto mod = [](int i, int j) { return i%j; }

((lambda) &) @0x7f16d19ba038


In [15]:
struct divide {
    int operator() (int denominator, int divisor) {
        return denominator / divisor;
    }
}

In [10]:
divide()(6, 3)

(int) 2


6.1 lambda
* lambda is also function object
* [  ] (arguments list) mnutable throw() -> bool {}
* [  ] for closure
    * [&] capture all by reference
    * [=] capture all by value
    * [n] capture variable n by value
    * [&n] capture variable by reference

In [4]:
{
    int n = 5;
    vector<int> numCount = {1,2,3,4,5};
    find_if(numCount.begin(), numCount.end(), [n](int a)->bool{return a==n;});
}

6.2 Function Object

In [12]:
struct StringPtrLess{
    bool operator() (const string *p1, const string *p2) const{
        return *p1 < *p2;
    }
}

In [20]:
{
    string a = "abc", b = "ABC";
    string *pa=&a, *pb=&b;
    cout << StringPtrLess(pa, pb) << endl;
}


?   


2) 通过类做闭包

6.3 bind

* bind 1st parameter

In [7]:
function<int(int)> plusOne = bind(add, 1, placeholders::_1)

(std::function<int (int)> &) @0x7fee6eb24018


* bind 2nd parameter

In [12]:
function<int(int)> modTwo = bind(mod, placeholders::_1, 2)

(std::function<int (int)> &) @0x7fee6eb24040


* bind parameter for function object

In [18]:
auto divideTwo = bind(divide(), placeholders::_1, 2)

(std::_Bind<divide (_Placeholder<1>, int)> &) @0x7fee6eb24060


In [19]:
divideTwo(6)

(int) 3


* bind class member function

closure provided by class instance as the first argument to class function (similar to this pointer)

In [2]:
class ToBindObj {
private:
    int id;

public:
    ToBindObj(int id):id(id){}
    void hello(const std::string& s){
        std::cout << "Hello " << id << " " << s << std::endl;
    }
}

?   


In [3]:
ToBindObj instance(10);
auto bindedHello = bind(&ToBindObj::hello, instance, "BINDED");

In [4]:
bindedHello()

Hello 10 BINDED


* bind static method

static method 

In [5]:
class ToBindStaticMethod {
public:
    static void hello(const std::string& s){
        std::cout << "Hello Static method " << s << std::endl;
    }
}

In [6]:
ToBindStaticMethod bindedStatic;
auto bindedHelloStatic = bind(ToBindStaticMethod::hello, "Binded");

In [7]:
bindedHelloStatic()

Hello Static method Binded


6.4 function

* function<int(int, int)>:  调用形式 call signature

In [11]:
function<int(int, int)> f1 = add;

* 可以把不对的函数对象用统一的call signature引用，可以理解为为函数定义类型

In [12]:
map<string, function<int(int, int)>> binops = {
    {"+", add},
    {"%", mod}
}

(std::map<std::string, std::function<int (int, int)> > &) { "%" => @0x8adb148, "+" => @0x880c9c8 }


In [13]:
binops["+"](1, 1)

(int) 2
